# Using enterprise to analyze PTA data

In this notebook you will learn:
* How to use `enterprise` to interact with NANOGrav data,
* How to setup an analysis of individual pulsar white noise properties,
* How to post-process your results.

If you are interested in working through this notebook, but do not want to install the software, we have prepared a [Google Colab notebook](https://colab.research.google.com/drive/11aRVepxn_whRm_JWCbgL_sVqn1hjo9Ik?usp=sharing)

By copying this notebook, you can install the software to your own Google Colab account and run the software without installation on your computer.

# Load packages and modules

In [1]:
from __future__ import division

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import precession_model

Optional mpi4py package is not installed.  MPI support is not available.


## Get par, tim, and noise files

In [2]:
psrlist = None # define a list of pulsar name strings that can be used to filter.

In [3]:
datadir = './data'
if not os.path.isdir(datadir):
    datadir = '../data'
print(datadir)

../data


In [4]:
psrstring = 'B1937+21'  # name of the pulsar

parfiles = sorted(glob.glob(os.path.join(os.path.join(datadir, 'par'), psrstring + '*')))
timfiles = sorted(glob.glob(os.path.join(os.path.join(datadir, 'tim'), psrstring + '*')))

# filter
if psrlist is not None:
    parfiles = [x for x in parfiles if x.split('/')[-1].split('.')[0] in psrlist]
    timfiles = [x for x in timfiles if x.split('/')[-1].split('.')[0] in psrlist]

# Make sure you use the tempo2 parfile for J1713+0747!!
# ...filtering out the tempo parfile... 
parfiles = [x for x in parfiles if 'J1713+0747_NANOGrav_12yv3.gls.par' not in x]

In [5]:
# if these are empty, then your data directory is wrong.
print(parfiles)
print(timfiles)

['../data/par/B1937+21_NANOGrav_12yv3.gls.par']
['../data/tim/B1937+21_NANOGrav_12yv3.tim']


## Load into `Pulsar` class list

* The `enterprise` Pulsar class uses `libstempo` to read in `par` and `tim` files, then stores all pulsar data into a `Pulsar` object. This object contains all data and meta-data needed for the ensuing pulsar and PTA analysis. You no longer need to reference the `par` and `tim` files after this cell.
* Note below that you can explicitly declare which version of the JPL solar-system ephemeris model that will be used to compute the Roemer delay between the geocenter and the barycenter (e.g. `DE438`). Otherwise the default values will be taken from the `par` files. Explicitly declaring the version here is good practice.
* You can also explicitly set the clock file to a version of `BIPM`, e.g. `BIPM(2018)`. This is less important, and you can let the code take the value from the `par` file.
* When you execute the following cell, you will get warnings like `WARNING: Could not find pulsar distance for PSR ...`. Don't worry! This is expected, and fine. Not all pulsars have well constrained distances, and will be set to `1 kpc` with a `20%` uncertainty.

### Read par and tim files into `enterprise` `Pulsar` objects

In [6]:
psrs = []
for p, t in zip(parfiles, timfiles):
    if psrstring in p:
        psr = Pulsar(p, t, ephem='DE438', clk='BIPM(2018)')
        psrs.append(psr)

[tempo2Util.C:396] Warning: [MISC1] Unknown parameter in par file:  DMX
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.

[tempo2Util.C:396] Warning: [PAR2] Have not set a DM epoch. The period epoch will be used instead. ../par/B1937+21_NANOGrav_12yv3.gls.par
[tempo2Util.C:396] Warning: [TIM1] Please place MODE flags in the parameter file 
[observatory.C:272] Warning: Assuming site '1' means 'gbt'
[observatory.C:272] Warning: Assuming site '3' means 'ao'


In [7]:
## Get parameter noise dictionary (for comparison in this notebook)
noise_ng12 = datadir + '/channelized_12p5yr_v3_full_noisedict.json'

params = {}
with open(noise_ng12, 'r') as fp:
    params.update(json.load(fp))

# Single pulsar analysis
* `enterprise` is structured so that one first creates `parameters`, then `signals` that these `parameters` belong to, then finally a `model` that is the union of all `signals` and the `data`.
* We will show this explicitly below, then introduce some model shortcut code that will make your life easier.
* We test on `J1853+1303`.

In [8]:
psr = [p for p in psrs if p.name == 'B1937+21'][0]

In [9]:
# find the maximum time span to set red-noise/DM-variation frequency sampling
tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = np.max(tmax) - np.min(tmin)
print(Tspan / 365.25 / 24 / 60 / 60)  # time span of data in years

12.772376976266186


In [10]:
# define selection by observing backend
# there will be separate white noise parameters for each observing backend
# since NANOGrav began taking data, there have been two generations of backends
# (ASP and PUPPI at Arecibo, GASP and GUPPI at Green Bank)
selection = selections.Selection(selections.by_backend)

## Create parameters
* White noise parameters are varied for each pulsar and then set to their most likely values when running the analyses for the entire array.
* These parameters are saved as a dictionary to a `.json` noisefile or set of noisefiles for easy access.
* Here we go through what one of these white noise searches looks like in long form, and then we will use shortcuts to do the same thing.

In [11]:
# white noise parameters
white_vary = True
if white_vary:
    efac = parameter.Uniform(0.01, 10.0)
    equad = parameter.Uniform(-8.5, -5)
    ecorr = parameter.Uniform(-8.5, -5)
else:
    efac = parameter.Constant() 
    equad = parameter.Constant() 
    ecorr = parameter.Constant() # we'll set these later with the params dictionary

# red noise parameters
log10_A = parameter.Uniform(-20, -11)
gamma = parameter.Uniform(0, 7)

## Create signals

In [12]:
# white noise
ef = white_signals.MeasurementNoise(efac=efac, log10_t2equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

# red noise (powerlaw with 30 frequencies)
# pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
# rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)
rn = precession_model.RedNoise_delay_block()

# timing model
tm = gp_signals.TimingModel(use_svd=True) # stabilizing timing model design matrix with SVD

## Piece the full model together

In [13]:
# full model
s = ef + ec + rn + tm


In [14]:
# intialize a single-pulsar pta model
# see how the "model" acts on the "pulsar" object...
pta = signal_base.PTA(s(psr))

In [15]:
pta.param_names

['B1937+21_L-wide_ASP_efac',
 'B1937+21_L-wide_ASP_log10_ecorr',
 'B1937+21_L-wide_ASP_log10_t2equad',
 'B1937+21_L-wide_PUPPI_efac',
 'B1937+21_L-wide_PUPPI_log10_ecorr',
 'B1937+21_L-wide_PUPPI_log10_t2equad',
 'B1937+21_Rcvr1_2_GASP_efac',
 'B1937+21_Rcvr1_2_GASP_log10_ecorr',
 'B1937+21_Rcvr1_2_GASP_log10_t2equad',
 'B1937+21_Rcvr1_2_GUPPI_efac',
 'B1937+21_Rcvr1_2_GUPPI_log10_ecorr',
 'B1937+21_Rcvr1_2_GUPPI_log10_t2equad',
 'B1937+21_Rcvr_800_GASP_efac',
 'B1937+21_Rcvr_800_GASP_log10_ecorr',
 'B1937+21_Rcvr_800_GASP_log10_t2equad',
 'B1937+21_Rcvr_800_GUPPI_efac',
 'B1937+21_Rcvr_800_GUPPI_log10_ecorr',
 'B1937+21_Rcvr_800_GUPPI_log10_t2equad',
 'B1937+21_S-wide_ASP_efac',
 'B1937+21_S-wide_ASP_log10_ecorr',
 'B1937+21_S-wide_ASP_log10_t2equad',
 'B1937+21_S-wide_PUPPI_efac',
 'B1937+21_S-wide_PUPPI_log10_ecorr',
 'B1937+21_S-wide_PUPPI_log10_t2equad',
 'RedNoise_P',
 'RedNoise_a1',
 'RedNoise_a2',
 'RedNoise_k',
 'RedNoise_t0']

In [16]:
len(pta.params)  # the higher this number is, the longer the model will take to  
                 # sample and the more samples it will require

29

## Draw initial sample from model parameter space

In [17]:
x0 = np.hstack([p.sample() for p in pta.params])
ndim = len(x0)

## Setup sampler (simple, with no tricks)

In [18]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution

# where chains will be written to
outdir = '../chains_singlepsr_wn_{}/'.format(str(psr.name))

# sampler object
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,
                 outDir=outdir, 
                 resume=False)

## Sample the parameter space

This will take around an **hour**. Note that the normal runs typically use 5e6 samples. For the sake of finishing the sampling process more quickly, we use a tenth of that for this tutorial. The sampling process can be interrupted and resumed at a later time by changing to `resume=True` in the above cell.

In [ ]:
# sampler for N steps
N = int(3e6) # normally, we would use 5e6 samples (this will save time)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

/home/wzy/miniconda3/envs/PTA/lib/python3.9/site-packages/enterprise/signals/parameter.py:62: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 0.33 percent in 1748.049000 s Acceptance rate = 0.210575Adding DE jump with weight 50
Finished 0.70 percent in 2013.177629 s Acceptance rate = 0.322095

/home/wzy/miniconda3/envs/PTA/lib/python3.9/site-packages/PTMCMCSampler/PTMCMCSampler.py:464: RuntimeWarning: All-NaN axis encountered
  np.nanmax([acor.acor(self._chain[self.burn : (iter - 1), ii])[0] for ii in range(self.ndim)]),


Finished 35.53 percent in 57758.634626 s Acceptance rate = 0.0664165

In [ ]:
print('finish')